In [ ]:
from PIL import Image
import glob
image_list = []
for filename in glob.glob('../input/men-women-classification/men/*jpg'): #assuming gif
    im=Image.open(filename)
    image_list.append(im)

In [ ]:
image_list2 = []
for filename in glob.glob('../input/men-women-classification/women/*jpg'): #assuming gif
    im2=Image.open(filename)
    image_list2.append(im2)

In [ ]:
import cv2
import numpy as np

In [ ]:
A = image_list2[0]

In [ ]:
B = image_list[0]

In [ ]:
A

In [ ]:
B

In [ ]:
A = A.resize((128,128))

In [ ]:
B = B.resize((128,128))

In [ ]:
A

In [ ]:
B

In [ ]:
A = np.array(A)

In [ ]:
A.shape

In [ ]:
import os
import pathlib
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image, ImageOps
from IPython.display import display
from sklearn.utils import shuffle
import warnings
from tensorflow.keras.utils import to_categorical
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
warnings.filterwarnings('ignore')

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,Activation
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [ ]:
X_man = np.zeros((600, 128, 128, 3), dtype=np.float32)
y_man = np.zeros((600,1), dtype=np.float32)
for i in range(600):
    img = image_list[i]
    img = img.resize((128,128))
    img = np.array(img)
    orig_img = img.astype(np.float32)/255.
    img = np.expand_dims(img, axis=0)
    X_man[i] = orig_img

In [ ]:
X_girl = np.zeros((600, 128, 128, 3), dtype=np.float32)
y_girl = np.full((600,1), 1)
for i in range(600):
    img = image_list2[i]
    img = img.resize((128,128))
    img = np.array(img)
    orig_img = img.astype(np.float32)/255.
    img = np.expand_dims(img, axis=0)
    X_girl[i] = orig_img

In [ ]:
X = np.concatenate((X_man, X_girl), axis=0)
y = np.concatenate((y_man, y_girl), axis=0)

In [ ]:
F1=  np.zeros((len(X), 128, 128, 3), dtype=np.float32)
F2=  np.zeros((len(y), 128, 128, 3), dtype=np.float32)

In [ ]:
for i in range(600):
    F1[i] = np.flipud(X[i]) 

In [ ]:
X = np.concatenate((X, F1), axis=0)

In [ ]:
for i in range(600):
    F2[i] = np.fliplr(X[i]) 

In [ ]:
X = np.concatenate((X, F2), axis=0)

In [ ]:
y = np.concatenate((y, y,y), axis=0)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.applications import Xception # TensorFlow ONLY
import numpy as np
import argparse
import cv2

In [ ]:
#i chouse xception algolithm but the shep i need to resize it to 71*71 that is minimum of this model
img_width, img_height = 128, 128

Xce_model = Xception(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [ ]:
def build_finetune_model_Xce(Xce_model, dropout, fc_layers, num_classes):
    # Freeze Parameters for train
    for layer in Xce_model.layers:
        layer.trainable = False

    x = Xce_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model_Xce = Model(inputs=Xce_model.input, outputs=predictions)

    return finetune_model_Xce

In [ ]:
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model_Xce = build_finetune_model_Xce(Xce_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=2)

In [ ]:
print(finetune_model_Xce.summary())

In [ ]:
from tensorflow.keras.optimizers import SGD
# Compile the model
epochs = 30
lrate = 0.001
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
finetune_model_Xce.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [ ]:
y_train.shape

In [ ]:
y.shape

In [ ]:
# Train the model
finetune_model_Xce.fit(X_train, y_train,
              batch_size=512,
              shuffle=True,
              epochs=epochs,
              validation_data=(X_test, y_test),
              callbacks=[EarlyStopping(min_delta=0.001, patience=3)])

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
# Predict the values from the validation dataset
Y_pred = finetune_model_Xce.predict(X_test)


# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
print(Y_pred_classes)
Y_true = np.argmax(y_test,axis = 1)
print(Y_true)
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()
print(accuracy_score(Y_true, Y_pred_classes))

In [ ]:
image_list3 = []
for filename in glob.glob('../input/boizer/boytest/*jpg'): #assuming gif
    im3=Image.open(filename)
    image_list3.append(im3)

In [ ]:
Image.fromarray(np.uint8(image_list3[0]))

In [ ]:
Image.fromarray(np.uint8(image_list3[1]))

In [ ]:
Image.fromarray(np.uint8(image_list3[2]))

In [ ]:
Image.fromarray(np.uint8(image_list3[3]))

In [ ]:
Image.fromarray(np.uint8(image_list3[4]))

In [ ]:
Image.fromarray(np.uint8(image_list3[5]))

In [ ]:
X_pre = np.zeros((6, 128, 128, 3), dtype=np.float32)
for i in range(6):
    img = image_list[i]
    img = img.resize((128,128))
    img = np.array(img)
    orig_img = img.astype(np.float32)/255.
    img = np.expand_dims(img, axis=0)
    X_pre[i] = orig_img

In [ ]:
Y_pred = finetune_model_Xce.predict(X_pre)


# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
print(Y_pred_classes)